## Get Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit
import sys
sys.path.append('../..')

# gather_keys_oauth2.py file needs to be in the same directory. 
# also needs to install cherrypy: https://pypi.org/project/CherryPy/
# pip install CherryPy
import src.data.gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime
from src import getSecret

CLIENT_ID=getSecret('FITBIT_CLIENT_ID')
CLIENT_SECRET=getSecret('FITBIT_CLIENT_SECRET')

server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()

ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[03/Apr/2023:12:32:06] ENGINE Listening for SIGTERM.
[03/Apr/2023:12:32:06] ENGINE Listening for SIGHUP.
[03/Apr/2023:12:32:06] ENGINE Listening for SIGUSR1.
[03/Apr/2023:12:32:06] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[03/Apr/2023:12:32:06] ENGINE Started monitor thread 'Autoreloader'.
[03/Apr/2023:12:32:06] ENGINE Serving on http://127.0.0.1:8090
[03/Apr/2023:12:32:06] ENGINE Bus STARTED


state: jUI8dtKqP3ImwV4HsiUYpNpwfMsNTC code: 461eff4b8ea8c81ab58362d838828116e3c6f8b3 error: None
Received code: 461eff4b8ea8c81ab58362d838828116e3c6f8b3
127.0.0.1 - - [03/Apr/2023:12:32:08] "GET /?code=461eff4b8ea8c81ab58362d838828116e3c6f8b3&state=jUI8dtKqP3ImwV4HsiUYpNpwfMsNTC HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15"


[03/Apr/2023:12:32:09] ENGINE Bus STOPPING
[03/Apr/2023:12:32:09] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8090)) shut down
[03/Apr/2023:12:32:09] ENGINE Stopped thread 'Autoreloader'.
[03/Apr/2023:12:32:09] ENGINE Bus STOPPED
[03/Apr/2023:12:32:09] ENGINE Bus EXITING
[03/Apr/2023:12:32:09] ENGINE Bus EXITED
[03/Apr/2023:12:32:09] ENGINE Waiting for child threads to terminate...


In [7]:
heart_rate_data = auth2_client.intraday_time_series('activities/heart', detail_level='1sec', base_date='2023-04-02')
df_heartRate = pd.DataFrame(heart_rate_data['activities-heart-intraday']['dataset'])

In [8]:
display(df_heartRate)

,time,value
0,19:50:52,62
1,19:50:57,63
2,19:51:02,65
3,19:51:07,66
4,19:51:12,65
...,...,...
1823,23:58:21,61
1824,23:58:26,63
1825,23:58:41,63
1826,23:58:46,62


In [9]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_heartRate['time'], y=df_heartRate['value'], mode='lines', name='Heart Rate'))

In [32]:
from datetime import datetime, timedelta
startTime = datetime.now() - timedelta(days=7)
endTime = datetime.now()
date_list = []
df_list = []
allDates = pd.date_range(start=startTime, end = endTime)
for oneDate in allDates:
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')
    df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])
    date_list.append(oneDate)
    df_list.append(df)
    
final_df_list = []
for date, df in zip(date_list, df_list):
    if len(df) == 0:
        continue
    df.loc[:, 'date'] = pd.to_datetime(date)
    final_df_list.append(df)

df_heartRate = pd.concat(final_df_list, axis = 0)
df_heartRate['datetime'] = pd.to_datetime(df_heartRate['date']) +pd.to_timedelta(df_heartRate['time'])
display(df_heartRate)

,time,value,date,datetime
0,19:50:52,62,2023-04-02,2023-04-02 19:50:52
1,19:50:57,63,2023-04-02,2023-04-02 19:50:57
2,19:51:02,65,2023-04-02,2023-04-02 19:51:02
3,19:51:07,66,2023-04-02,2023-04-02 19:51:07
4,19:51:12,65,2023-04-02,2023-04-02 19:51:12
...,...,...,...,...
1584,12:33:14,62,2023-04-03,2023-04-03 12:33:14
1585,12:33:29,62,2023-04-03,2023-04-03 12:33:29
1586,12:33:34,61,2023-04-03,2023-04-03 12:33:34
1587,12:33:44,62,2023-04-03,2023-04-03 12:33:44


In [35]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_heartRate['datetime'], y=df_heartRate['value'], mode='lines', name='Heart Rate'))

In [36]:
oneDayData['activities-heart'][0]['value']['restingHeartRate']

50

## 7.) Resting Heart Rate

In [ ]:
# startTime is first date of data that I want. 
# You will need to modify for the date you want your data to start
startTime = pd.datetime(year = 2020, month = 1, day = 1)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

In [ ]:
date_list = []
resting_list = []

allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.intraday_time_series('activities/heart', base_date=oneDate, detail_level='1sec')
    
    date_list.append(oneDate)
    
    resting_list.append(oneDayData['activities-heart'][0]['value']['restingHeartRate'])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

ax.plot(date_list, resting_list )

# This is just making it so there isnt a grid line or text for every point
xtick_list = []
xticklabel_list = []
for index, label in enumerate(ax.get_xticklabels()):
    if index % 5 == 0:
        xticklabel_list.append(label)
        xtick_list.append(index)

ax.grid(True,
    axis = 'both',
    zorder = 0,
    linestyle = ':',
    color = 'k')
ax.tick_params(axis = 'both', labelsize = 20)
ax.set_xticks(xtick_list)
ax.tick_params(axis = 'x', rotation = 90,labelsize = 20)
ax.set_xlim(0, index)
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 45, rotation_mode="anchor", ha = 'right')
ax.set_xlabel('Date', fontsize = 24)
ax.set_ylabel('Resting Heart Rate', fontsize = 24)
fig.tight_layout()
fig.savefig('restingHR_graph.png', format = 'png', dpi = 300)

In [ ]:
resting_df = pd.DataFrame({'date': date_list, 'RHR': resting_list})

In [ ]:
resting_df.head()

## 8.) Get Sleep Data

In [ ]:
startTime = pd.datetime(year = 2020, month = 1, day = 5)
endTime = pd.datetime.today().date() - datetime.timedelta(days=1)

In [ ]:
allDates = pd.date_range(start=startTime, end = endTime)

In [ ]:
date_list = []
df_list = []
stages_df_list = []

allDates = pd.date_range(start=startTime, end = endTime)

for oneDate in allDates:
    
    oneDate = oneDate.date().strftime("%Y-%m-%d")
    
    oneDayData = auth2_client.sleep(date=oneDate)
    
    # get number of minutes for each stage of sleep and such. 
    stages_df = pd.DataFrame(oneDayData['summary'])

    df = pd.DataFrame(oneDayData['sleep'][0]['minuteData'])
    
    date_list.append(oneDate)
    
    df_list.append(df)
    
    stages_df_list.append(stages_df)
    
final_df_list = []

final_stages_df_list = []

for date, df, stages_df in zip(date_list, df_list, stages_df_list):

    if len(df) == 0:
        continue
    
    df.loc[:, 'date'] = pd.to_datetime(date)
    
    stages_df.loc[:, 'date'] = pd.to_datetime(date)
    
    final_df_list.append(df)
    final_stages_df_list.append(stages_df)

final_df = pd.concat(final_df_list, axis = 0)

final_stages_df = pd.concat(final_stages_df_list, axis = 0)

In [ ]:
columns = final_stages_df.columns[~final_stages_df.columns.isin(['date'])].values

In [ ]:
columns

In [ ]:
pd.concat([final_stages_df[columns] + 2, final_stages_df[['date']]], axis = 1)

In [ ]:

# Export file to csv
final_df.to_csv('minuteSleep' + '.csv', index = False)
final_stages_df.to_csv('minutesStagesSleep' + '.csv', index = True)